In [1]:
import matplotlib.pyplot as plt
import circle_fit as cf
import numpy as np
import cv2
import os
%run Functions.ipynb

## Countour

## 比較出現 pixel 的現象

In [9]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")

## 製作所有可能邊出現之地圖
file = "10_mean_0.bmp"
img = cv2.imread(file,0)
h,w = img.shape[:2]
Map = np.zeros((h,w,3))
images = os.listdir()
images.sort()

for image in images:
    img = cv2.imread(image,0)

    blur, nlm, otsu, edge = image_processing(img)
    edge_points = get_edge_points(edge)
    
    for x, y in edge_points:
        Map[y, x, :] = Map[y, x, :] + [0, 0, 1]
  
    
    
for i, Row in enumerate(Map[:,:,2]):
    for j, val in enumerate(Row):
        if (val >= 75):
            Map[i, j, :] = [0,0,255]
        elif (val >= 50):
            Map[i, j, :] = [0,118,238]
        elif (val >= 30):
            Map[i, j, :] = [0,215,255]
        elif (val >= 10):
            Map[i, j, :] = [34,139,34]
        elif (val >= 1):
            Map[i, j, :] = [255,0,0]
        else:
            Map[i, j, :] = [0,0,0]

cv2.imwrite("/home/mj/HardDisk/Github/NTU/ARCS/Demo/2020.01.03/10_Mean_Map.bmp", Map)
cv2.imshow("Map", Map)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
Map = cv2.imread("/home/mj/HardDisk/Github/NTU/ARCS/Demo/2020.01.03/Ori_Map.bmp")
cv2.imshow("Map", Map)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 比較不同 Resolution

In [40]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")
file = "10_mean_80.bmp"
img = cv2.imread(file,0)
h,w = img.shape[:2]

resized = cv2.resize(img, (w*2, h*2))
# enchanced = swift_algo(resized)
blur = cv2.GaussianBlur(resized, (11,11), 0)
nlm = cv2.fastNlMeansDenoising(blur, None, 10, 7, 21)
enchanced = swift_algo(nlm)
_, otsu = cv2.threshold(enchanced, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
edge = cv2.Canny(otsu, 0, 20)
edge_points = get_edge_points(edge)
cy,cx,r,_ = cf.least_squares_circle((edge_points))
print(cy/2)
print(cx/2)

resized = cv2.resize(img, (int(w/2), int(h/2)))
# enchanced = swift_algo(resized)
blur = cv2.GaussianBlur(resized, (3,3), 0)
nlm = cv2.fastNlMeansDenoising(blur, None, 10, 7, 21)
enchanced = swift_algo(nlm)
_, otsu = cv2.threshold(enchanced, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
edge = cv2.Canny(otsu, 0, 20)
edge_points = get_edge_points(edge)
cy,cx,r,_ = cf.least_squares_circle((edge_points))
print(cy*2)
print(cx*2)

cv2.imshow("Resizd", resized)
cv2.imshow("Enhanced", enchanced)
cv2.imshow("Blur", blur)
cv2.imshow("NLM", nlm)
cv2.imshow("Otsu", otsu)
cv2.imshow("Edge", edge)
cv2.waitKey(0)
cv2.destroyAllWindows()

203.73934499580767
99.73042279520683
202.29628722449291
98.89638363540156


## 測試擴增Edge 試圖建立虛擬連結

### Version 1 移動有問題的點

In [67]:
def move_edge(edited_edge, edge_points, img):
    
    edited_index = []
    for index, point in enumerate(edge_points):
        x, y = point[:]
        value = img[y,x]
        
        for i in range(-1, 1):
            for j in range(-1, 1):
                if int(value) - int(img[y+i,x+j]) > 2:
                    edited_edge.append([x+j, y+i])
                    edited_index.append(index)
    
    edited_edge = np.delete(edited_edge, edited_index, axis=0)

### Versoin 2 直接增加可能的點

In [68]:
def add_edge(edited_edge, edge_points, img):
    
    for index, point in enumerate(edge_points):
        x, y = point[:]
        value = img[y,x]
        
        for i in range(-1, 1):
            for j in range(-1, 1):
                if int(value) - int(img[y+i,x+j]) > 2:
                    edited_edge.append([x+j, y+i])
                    

### Version 3 直接去除問題點

In [69]:
def del_edge(edited_edge, edge_points, img):
    
    edited_index = []
    for index, point in enumerate(edge_points):
        x, y = point[:]
        value = img[y,x]
        
        for i in range(-1, 1):
            for j in range(-1, 1):
                if int(value) - int(img[y+i,x+j]) > 2:
                    edited_index.append(index)
    
    edited_edge = np.delete(edited_edge, edited_index, axis=0)

## 編輯邊緣

In [47]:
def edit_edge(edge_points, img, funct):
    edited_edge = edge_points.copy()
    
    if funct == 0:
        move_edge(edited_edge, edge_points, img)
    elif funct == 1: 
        add_edge(edited_edge, edge_points, img)
    else:
        del_edge(edited_edge, edge_points, img)
    
    edited_edge = np.unique(edited_edge, axis=0)
    return edited_edge

In [66]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")

file = "10_mean_80.bmp"
img = cv2.imread(file,0)
blur = cv2.GaussianBlur(img, (5,5), 0)
nlm = cv2.fastNlMeansDenoising(blur, None, 10, 7, 21)
_, otsu = cv2.threshold(nlm, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
edge = cv2.Canny(otsu, 0, 20)
edge_points = get_edge_points(edge)

# edited_edge = edit_edge(edge_points, img, funct=2)

print(len(edge_points))
# print(len(edited_edge))

442


## 測試多次隨機取子點擬合的分佈
假定平均 cx: 203.61381  cy: 99.17248
### 結論：無法單靠尋找 Fram 各自的 Median 或 多次 Random subsampled 做連結

In [50]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")

file = "10_mean_80.bmp"
img = cv2.imread(file,0)
edge = image_processing(img)
edge_points = get_edge_points(edge)


xs = []
ys = []
for iter in range(10):
    sampled_indices = np.random.choice(range(len(edge_points)), size=int(len(edge_points)), replace=False)
    sample_points = edge_points[sampled_indices]
#     cx,cy,r,_ = cf.least_squares_circle((sample_points))
    cx = np.median(sample_points[:,0])
    cy = np.median(sample_points[:,1])
    xs.append(cx)
    ys.append(cy)

print(xs)
print(ys)

[205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0]
[100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5]
